In [7]:
!pip install openai llama-index-llms-openai python-dotenv llama-index gradio > installs.txt

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [9]:
import gradio as gr
from llama_index.core import SQLDatabase
import openai
from openai import OpenAI as gpt
from llama_index.llms.openai import OpenAI
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select
from IPython.display import Markdown, display
from llama_index.core.query_engine import NLSQLTableQueryEngine
client = gpt()
def optimize_sql_query(query, openai_api_key, model="gpt-3.5-turbo", temperature=0, max_tokens=256, frequency_penalty=0, presence_penalty=0):
    system_message = "Given the following SQL query, analyze this query and suggest optimizations to improve performance:\n"
    user_message = query
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    openai.api_key = openai_api_key
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty
    )

    optimization_suggestions = response.choices[0].message.content.strip()
    
    return optimization_suggestions
def execute_query(tables, query_str):
    db_file_path = "company.db"
    engine = create_engine(f"sqlite:///{db_file_path}")
    llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
    sql_database = SQLDatabase(engine, include_tables=tables)
    query_engine = NLSQLTableQueryEngine(
        sql_database=sql_database, tables=tables, llm=llm,
    )
    response = query_engine.query(query_str)
    response_text = str(response)
    sql_query = response.metadata['sql_query']
    optimizations=optimize_sql_query(sql_query, api_key)
    return response_text, sql_query,optimizations

iface = gr.Interface(
    fn=execute_query,
    inputs=[
        gr.CheckboxGroup(["employees", "salaries", "projects", "departments"], label="Select tables"),
        gr.Textbox(lines=2, placeholder="Enter your query string"),
    ],
    outputs=[
        gr.Textbox(label="Response"),
        gr.Textbox(label="SQL Query"),
        gr.Textbox(label="Optimization suggestions")
    ],
    title="Natural Language to SQL Query Engine",
    description="Enter the table names, query string, and your SQLAlchemy engine string to execute the query.",
)

iface.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
